In [6]:
import pandas as pd

years = [23, 24, 25]

# unisci i dataframe di tutti gli anni in un unico dataframe con le dummies

df_final = pd.DataFrame()

for year in years:
    df = pd.read_csv(f"df_pre_dummies/df_{year}.csv")

    # drop_first=True to avoid collinearity during regression
    df = pd.get_dummies(df, columns=["EventType", "Surface"], prefix=["EventType", "Surface"], dtype=int, drop_first=True)

    df_final = pd.concat([df_final, df], ignore_index=True)

display(df_final)


,PlayerCode,EventId,iscritto,year,date_tournament,EventName,EventCountry,TotPrizeMoney,Same_Nationality,Rank,EventType_500,Surface_Grass,Surface_Hard
0,mv14,891,1,2023,2023-01-02,Pune,India,642735,0,50,0,0,1
1,mv14,2843,0,2023,2023-01-02,Adelaide 1,Australia,642735,0,50,0,0,1
2,mw02,891,0,2023,2023-01-02,Pune,India,642735,0,51,0,0,1
3,mw02,2843,0,2023,2023-01-02,Adelaide 1,Australia,642735,0,51,0,0,1
4,mc65,891,0,2023,2023-01-02,Pune,India,642735,0,52,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53551,n0db,496,0,2025,2025-02-10,Marseille,France,740730,0,601,0,0,1
53552,n0db,499,0,2025,2025-02-10,Delray Beach,"FL, U.S.A.",680140,0,601,0,0,1
53553,n0db,424,0,2025,2025-02-03,Dallas,"TX, U.S.A.",2760000,0,601,1,0,1
53554,n0db,6932,0,2025,2025-02-17,Rio de Janeiro,Brazil,2396115,0,601,1,0,0


In [11]:
import statsmodels.api as sm
import numpy as np

df_final_copy = df_final.copy()

y = df_final_copy["iscritto"]

X = df_final_copy.drop(columns=["iscritto", "PlayerCode", "EventId", "year", "date_tournament", "EventName", "EventCountry"])

#normalizza le variabili con StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_fin = scaler.fit_transform(X)
# rinomina le colonne con i nomi originali
X_fin = pd.DataFrame(X_fin, columns=X.columns)

X_fin = sm.add_constant(X_fin)
model = sm.Logit(y, X_fin)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.263033
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               iscritto   No. Observations:                53556
Model:                          Logit   Df Residuals:                    53549
Method:                           MLE   Df Model:                            6
Date:                Wed, 14 Jan 2026   Pseudo R-squ.:                  0.1412
Time:                        19:28:46   Log-Likelihood:                -14087.
converged:                       True   LL-Null:                       -16403.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -2.8278      0.023   -121.417      0.000      -2.873      -2.782
TotPrizeMon